In [ ]:
# Data ingestion in MLOps is the process of collecting, 
# importing, and preparing raw data from various sources 
# (databases, files, APIs, etc.) 
# so it can be used for machine learning workflows.
#  It is the first step in the ML pipeline, 
# ensuring that data is available, accessible, and in the right format
#  for further processing, training, and evaluation.
#  Data ingestion often includes tasks like validation, cleaning, and 
# transformation.

In [2]:
import os


In [3]:
%pwd

'c:\\Project\\MLops_basics_project\\research'

In [4]:
os.chdir('../')

In [16]:
import os
from box.exceptions import BoxValueError
import yaml
from mlProject import logger
import json
import joblib
from ensure import ensure_annotations
from box import ConfigBox
from pathlib import Path
from typing import Any

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path


In [8]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml,create_directories

c:\project\mlops_basics_project\src\mlProject\constants\__init__.py:2: SyntaxWarning: invalid escape sequence '\P'
  CONFIG_FILE_PATH=Path('C:\Project\MLops_basics_project\config\config.yaml')
c:\project\mlops_basics_project\src\mlProject\constants\__init__.py:3: SyntaxWarning: invalid escape sequence '\P'
  PARAMS_FILE_PATH=Path('C:\Project\MLops_basics_project\params.yaml')
c:\project\mlops_basics_project\src\mlProject\constants\__init__.py:4: SyntaxWarning: invalid escape sequence '\P'
  SCHEMA_FILE_PATH=Path('C:\Project\MLops_basics_project\schema.yaml')


In [10]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self)->DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config=DataIngestionConfig(root_dir=config.root_dir,
                                                  source_URL=config.source_URL,
                                                  local_data_file=config.local_data_file,
                                                  unzip_dir=config.unzip_dir)
        return data_ingestion_config


In [11]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size


In [13]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers=request.urlretrieve(url=self.config.source_URL,
                                                 filename=self.config.local_data_file)
            logger.info(f"{filename} downloaded with following info:\n {headers}")
        else:
            logger.info(f"file already exsits  of size {get_size(Path(self.config.local_data_file))}")
    def extract_zip_file(self):
        unzip_path=self.config.unzip_dir
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            
        

In [14]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
   raise e
    

[2025-07-28 23:39:49,517: INFO: common: yaml file: C:\Project\MLops_basics_project\config\config.yaml loaded successfully]
[2025-07-28 23:39:49,522: INFO: common: yaml file: C:\Project\MLops_basics_project\params.yaml loaded successfully]
[2025-07-28 23:39:49,524: INFO: common: yaml file: C:\Project\MLops_basics_project\schema.yaml loaded successfully]
[2025-07-28 23:39:49,525: INFO: common: created directory at: artifacts]
[2025-07-28 23:39:49,527: INFO: common: created directory at: artifacts/data_ingestion]
[2025-07-28 23:39:49,529: INFO: 1371221901: file already exsits  of size ~ 23 KB]
